In [ ]:
pip install requests beautifulsoup4 pandas


In [ ]:
!pip install selenium


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

def fetch_recipe_data_selenium(url):
    options = Options()
    options.headless = True  # Headless mode, no browser window

    # Don't use user-data-dir to avoid session conflicts
    # options.add_argument(f'--user-data-dir={temp_user_data_dir}')  # REMOVE THIS

    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        print(f"Loading {url} ...")
        time.sleep(7)  # wait for page to load fully

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        title_tag = soup.find('h1', class_='headline heading-content')
        title = title_tag.get_text(strip=True) if title_tag else "No title found"

        ingredients_tags = soup.select('li.ingredients__item')
        ingredients = [tag.get_text(strip=True) for tag in ingredients_tags] if ingredients_tags else []

        instructions_tags = soup.select('li.instructions-section-item div.section-body')
        instructions = [tag.get_text(strip=True) for tag in instructions_tags] if instructions_tags else []

        return {
            'Title': title,
            'Ingredients': ', '.join(ingredients),
            'Instructions': ' '.join(instructions)
        }
    finally:
        driver.quit()

if __name__ == "__main__":
    urls = [
        'https://www.epicurious.com/recipes/food/views/spaghetti-carbonara-51217710',
        'https://www.epicurious.com/recipes/food/views/chocolate-chip-cookies-51106320',
        'https://www.epicurious.com/recipes/food/views/classic-deviled-eggs-56390335',
        'https://www.epicurious.com/recipes/food/views/easy-roast-chicken-56390113',
        'https://www.epicurious.com/recipes/food/views/banana-bread-51150870',
    ]

    all_recipes = []
    for url in urls:
        print(f"Fetching: {url}")
        try:
            data = fetch_recipe_data_selenium(url)
            if data:
                all_recipes.append(data)
        except Exception as e:
            print(f"Failed to fetch {url} due to error: {e}")

    if all_recipes:
        df = pd.DataFrame(all_recipes)
        df.to_csv('epicurious_recipes.csv', index=False)
        print("Data saved to 'epicurious_recipes.csv'")
    else:
        print("No recipes fetched.")
